In [0]:
dbutils.widgets.text("collection_name", "", "Collection Name")
dbutils.widgets.text("output_path", "", "Output Path")
dbutils.widgets.text("database_name", "", "Database Name")

In [0]:
import json
import logging
import uuid

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure

In [0]:
database_name = dbutils.widgets.get("database_name")
collection_name = dbutils.widgets.get("collection_name")
output_path = dbutils.widgets.get("output_path")


In [0]:
uri = dbutils.secrets.get(scope="crossy", key="mongodb_uri")

In [0]:
# Create output directory if it doesn't exist
dbutils.fs.mkdirs(output_path)
print(output_path)


/Volumes/stafford_1_classic_catalog/crossy/mongodb_output/incremental


In [0]:
print(f"Connecting to MongoDB")
client = MongoClient(uri)
        
# Test connection
client.admin.command('ping')
print("Successfully connected to MongoDB")       
        

Connecting to MongoDB
Successfully connected to MongoDB


In [0]:
# Get database and collection
db = client[database_name]
collection = db[collection_name]
        
# Check if collection exists
if collection_name not in db.list_collection_names():
    print(f"Collection '{collection_name}' does not exist in database '{database_name}'")
else:
    print(f"Collection '{collection_name}' does exist in database '{database_name}'")
        

        

Collection 'vehicle' does exist in database 'crossy'


In [0]:
# Open a change stream for the collection
with collection.watch(full_document='updateLookup') as stream:
    for change in stream:
        uuid_str = str(uuid.uuid4())
        doc_id = str(change["documentKey"]["_id"]) + uuid_str
        file_path = f"{output_path}/{doc_id}.json"
        with open(file_path, "w") as f:
            json.dump(change, f, default=str)
        print(file_path)

/Volumes/stafford_1_classic_catalog/crossy/mongodb_output/incremental/693f63e58c792d57fa893b5696ebd6d0-c5f5-45f2-b3d4-32d60698dd87.json
/Volumes/stafford_1_classic_catalog/crossy/mongodb_output/incremental/693d817a8c792d57fa893b53e405fefc-08b0-4d85-8951-5f9afc2696ce.json


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can